In [103]:
from google.colab import drive

In [104]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [105]:
!ls -lha1 ./drive/MyDrive/reduced_images_ILB.zip

-rw------- 1 root root 1011M Mar  9 18:30 ./drive/MyDrive/reduced_images_ILB.zip


In [106]:
!unzip ./drive/MyDrive/reduced_images_ILB.zip -d . > /dev/null

replace ./reduced_images/test/ann_11794173/29d23384843309ad1054e2a38c63631cf49996ae.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [107]:
import pandas as pd
import numpy as np
from PIL import Image

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import transforms

In [108]:
PATH_TABULAR_DATA = "./drive/MyDrive/real_estate_price_prediction_processed_09042024.csv"

In [109]:
data_tab = pd.read_csv(PATH_TABULAR_DATA)

In [110]:
data_tab[["price_scaled"]] = MinMaxScaler().fit_transform(data_tab[["price"]])

In [111]:
data_tab.head()

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,price,price_scaled
0,35996577,0,43.643880,7.117183,8452,199,63.0,3.479524,3995.665362,205.385148,...,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,355000.0,0.145320
1,35811033,0,45.695757,4.895610,8234,3347,90.0,3.000000,3995.665362,223.000000,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,190000.0,0.072778
2,35731841,12,47.966791,-1.220451,5235,1650,61.0,3.479524,370.000000,205.385148,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39000.0,0.006390
3,35886765,12,47.289292,-1.878805,1929,2050,142.0,3.479524,764.000000,217.000000,...,8.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,299000.0,0.120699
4,35781137,0,45.718992,4.844234,4476,3334,88.0,3.000000,3995.665362,205.385148,...,5.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,478000.0,0.199397


In [112]:
X_train_, X_test, y_train_, y_test= train_test_split(
    data_tab.drop(columns=["price", "price_scaled"]),
    data_tab["price_scaled"],
    test_size=0.20,
    random_state=123,
    shuffle=True
    )

In [113]:
X_train, X_val, y_train, y_val = train_test_split(
    X_train_,
    y_train_,
    test_size=0.20,
    random_state=123,
    shuffle=True
    )

In [114]:
X_reorganized = pd.concat(
    [
        pd.DataFrame(X_train).assign(group="train"),
        pd.DataFrame(X_val).assign(group="val"),
        pd.DataFrame(X_test).assign(group="test")
    ]
).reset_index(drop=True)


y_reorganized = pd.concat(
    [
        pd.DataFrame(y_train).assign(group="train"),
        pd.DataFrame(y_val).assign(group="val"),
        pd.DataFrame(y_test).assign(group="test")
    ]
).reset_index(drop=True)

In [115]:
X_reorganized.sample(5)

,id_annonce,property_type,approximate_latitude,approximate_longitude,city,postal_code,size,floor,land_size,energy_performance_value,...,nb_boxes,nb_photos,has_a_balcony,nb_terraces,has_a_cellar,has_a_garage,has_air_conditioning,last_floor,upper_floors,group
7552,35859815,0,43.220701,2.351864,1296,363,6047.0,3.000000,3995.665362,205.385148,...,0.0,13.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,train
18658,35897151,12,48.548150,3.299115,6044,3745,90.0,3.479524,999.000000,205.385148,...,1.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
13111,35750693,12,42.698983,2.896937,5690,3165,1401.0,3.479524,448.000000,205.385148,...,0.0,17.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,train
14079,35842439,0,43.666479,1.285011,4954,1425,42.0,3.479524,3995.665362,205.385148,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,train
24301,35864507,12,47.215657,-1.737273,1978,2037,92.0,3.479524,3995.665362,205.385148,...,0.0,5.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,val


In [116]:
#X_reorganized[["id_annonce", "group"]]

In [117]:
PATH_IMAGES_DATA = "./drive/MyDrive/real_estate_price_prediction_metadata_images_09042024.csv"

In [118]:
data_img = pd.read_csv(PATH_IMAGES_DATA)

In [119]:
data_img_agg = data_img.groupby('id_annonce')['long_path_image'].apply(list).reset_index(name='images')

In [120]:
data_img_groups = data_img_agg.merge(
    X_reorganized[["id_annonce", "group"]],
    how='left',
    on='id_annonce'
)

In [121]:
data_img_groups.sample(10)

,id_annonce,images,group
15658,35852177,[./reduced_images/train/ann_35852177/8b53cd176...,train
8386,35800323,[./reduced_images/train/ann_35800323/e71238f72...,train
17306,35862333,[./reduced_images/train/ann_35862333/6f2f7381c...,train
28992,36018019,[./reduced_images/train/ann_36018019/3bbaa2fc2...,train
1457,35577773,[./reduced_images/train/ann_35577773/ef36e50e7...,train
19990,35879287,[./reduced_images/train/ann_35879287/29fd62a91...,train
27778,36010037,[./reduced_images/train/ann_36010037/099fc6589...,test
19176,35874465,[./reduced_images/train/ann_35874465/ebbd27679...,test
22969,35906783,[./reduced_images/train/ann_35906783/83a50f598...,train
12980,35834053,[./reduced_images/train/ann_35834053/b0a955f45...,val


In [122]:
# Define the image transformation
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        ),
])


In [123]:
class CustomDataset(Dataset):

    def __init__(
        self,
        data_features,
        data_target,
        data_img_groups,
        group='train',
        transform=None
        ) -> None:

        self.group = group

        self.data_features = data_features.query(
            f"group == '{self.group}'"
            )

        self.data_target = data_target.query(
            f"group == '{self.group}'"
            )

        self.data_img_groups = data_img_groups.query(
            f"group == '{self.group}'"
            )

        self.transform = transform

        self.model = models.resnet152(pretrained=True)
        self.model_layers = list(self.model.children())[:-1]
        self.model_feature_extract = torch.nn.Sequential(*self.model_layers)

        self.set_model_eval()

    def __len__(self):
        return len(self.data_features)

    def set_model_eval(self):
      """
      Set to evaluation model.
      """
      return self.model.eval()


    def get_model_embedding(self, img_tensor):

      with torch.no_grad():
          # Get the image features from the ResNet-152 model
          img_features = self.model_feature_extract(img_tensor)

      return img_features

    def load_image(self, path) -> Image.Image:

        return Image.open(path).convert("RGB")

    def __getitem__(self, index):

        images_path = self.data_img_groups.iloc[index]["images"]

        images_emmbeded = []

        for path in images_path:
          image = self.load_image(path)

          if self.transform:
            image = self.transform(image).unsqueeze(0)

            img_embedding = self.get_model_embedding(image)
            images_emmbeded.append(img_embedding.flatten())

        #images_embbeding = torch.mean(
        #                  torch.Tensor(np.array(images_emmbeded)),
        #                  axis = 0
        #                  )

        images_embbeding = torch.stack(images_emmbeded).mean(dim=0)

        features = self.data_features.drop(
            columns=["id_annonce","group"]
            ).iloc[index,:]

        #features = torch.from_numpy(features).float()

        target = self.data_target[["price_scaled"]].values[index]
        #target = np.array(target)
        #target = torch.from_numpy(target).float()

        #sample = {}

        avg_embbeding = images_embbeding
        tab_features = torch.tensor(features.values, dtype=torch.float32)
        target = torch.tensor(target, dtype=torch.float32)

        return avg_embbeding, tab_features, target

In [124]:
train_dataset = CustomDataset(
        data_features=X_reorganized,
        data_target=y_reorganized,
        data_img_groups=data_img_groups,
        group='train',
        transform=preprocess
    )

val_dataset = CustomDataset(
        data_features=X_reorganized,
        data_target=y_reorganized,
        data_img_groups=data_img_groups,
        group='val',
        transform=preprocess
)

test_dataset = CustomDataset(
        data_features=X_reorganized,
        data_target=y_reorganized,
        data_img_groups=data_img_groups,
        group='test',
        transform=preprocess
)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [125]:
BATCH_SIZE = 1024

In [126]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [127]:
# Define the neural network
class PricePredictionModel(nn.Module):
    def __init__(self, embedding_size=2048, tabular_input_size=26):  # Adjust tabular_input_size based on your actual features
        super(PricePredictionModel, self).__init__()
        self.fc1 = nn.Linear(embedding_size+tabular_input_size, 512)  # Output a single value for price prediction
        self.fc2 = nn.Linear(512, 1)

    def forward(self, image_inputs, tabular_inputs):
        combined_embedding = torch.cat((image_inputs, tabular_inputs), dim=1)
        output = self.fc1(combined_embedding)
        output = self.fc2(output)
        return output.squeeze(1)

In [128]:
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"

In [129]:
device

'cpu'

In [130]:
#model = MultiModal(device=device)

In [131]:
# Initialize the model, loss, and optimizer
model = PricePredictionModel().to(device)

In [132]:
class EarlyStopping:
    def __init__(self, patience, verbose=False, delta=0, path="checkpoint.pt"):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        return self.early_stop

    def save_checkpoint(self, val_loss, model):
        """Saves model when validation loss decrease."""
        if self.verbose:
            print(
                f"Validation Loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model to {self.path}"
            )
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

    def load_checkpoint(self, model):
        """Loads the best model weights from the saved checkpoint."""
        model.load_state_dict(torch.load(self.path))

In [133]:
# Tunners
LR = 0.001
EPOCHS = 1

In [134]:
optimizer = optim.Adam(model.parameters(), lr=LR, amsgrad=True, weight_decay=LR * 0.1)

criterion = nn.L1Loss().to(device) #nn.L1Loss().to(device)

early_stopping = EarlyStopping(patience=int(EPOCHS * 0.5), verbose=True)

In [135]:
def train_eval(model, loader, optimizer, loss_func, is_training):
    loss_list = []
    stop = False
    model.train() if is_training else model.eval()

    with torch.set_grad_enabled(is_training):  # Enable gradients only in training
        for batch_idx, (avg_embbeding, tab_features, target) in tqdm(
            enumerate(loader),
            desc="Epoch_Train" if is_training else "Epoch_Val",
            total=len(loader),
        ):
            avg_embbeding, tab_features, target = avg_embbeding.to(device), tab_features.to(device), target.to(device)

            if is_training:
                optimizer.zero_grad()  # Reset gradients
            output = model(avg_embbeding, tab_features)
            loss = loss_func(output, target.squeeze(1))

            if is_training:
                loss.backward()  # Compute gradients
                optimizer.step()  # Update weights

            loss_list.append(loss.item())

    if not is_training:
        stop = early_stopping(
            (sum(loss_list) / len(loss_list)), model
        )  # Early stopping decision

    return loss_list, stop

In [136]:
def calc_acc_loss(total_loss, loss):
    total_loss.append(sum(loss) / len(loss))

    #total_accuracy.append(100 * correct / len(loader.dataset))
    return total_loss

In [137]:
# Lists
total_loss_train = []
total_loss_val = []
total_accuracy_train = [1]
total_accuracy_val = [1]

In [138]:
torch.cuda.empty_cache()
try:
  torch.cuda.reset_peak_memory_stats()
except:
  pass

In [ ]:
%%time
# Training and Evaluation loop
for epoch in range(EPOCHS):
    # Train Iteration
    loss_train, _ = train_eval(
        model, train_loader, optimizer, criterion, is_training=True
    )

    # Val Iteration
    loss_val = train_eval(
        model, val_loader, optimizer, criterion, is_training=False
    )

    # Calculate and record loss & accuracy for training and validation
    total_loss_train, total_accuracy_train = calc_acc_loss(
        total_loss_train,
        loss_train
    )

    # Calculate Loss and Accuracy
    total_loss_val, stop = calc_acc_loss(
        total_loss_val,
        loss_val
    )

    # Print metrics per epoch
    print(
        f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {total_loss_train[-1]:.4f}, Val Loss: {total_loss_val[-1]:.4f}"
    )

    if stop:
        print("Early Stopping")
        break

Epoch_Train:   0%|          | 0/24 [00:00<?, ?it/s]

## Texting

In [ ]:
try:
  torch.cuda.empty_cache()
  torch.cuda.reset_peak_memory_stats()
except:
  pass

In [ ]:
model_pred = PricePredictionModel().to(device) #MultiModal(device=device)
model_pred.load_state_dict(torch.load("checkpoint.pt"))
model_pred.to(device)

In [ ]:
def test(model, loader):
    model.eval()
    total_loss_test = []
    outputs_test = []
    target_test = []

    pred = torch.Tensor().to(device)

    with torch.no_grad():
        for batch_idx, (avg_embbeding, tab_features, target) in tqdm(
            enumerate(loader), desc="Test Epoch", total=len(loader)
        ):
            avg_embbeding, tab_features, target = avg_embbeding.to(device), tab_features.to(device), target.to(device)
            output = model(avg_embbeding, tab_features)

            outputs_test.append(output)
            target_test.append(target)

            loss = criterion(output, target.squeeze(1))
            total_loss_test.append(loss.item())

        outputs_test = torch.concat(outputs_test, dim=0).numpy()
        target_test = torch.concat(target_test, dim=0).numpy()

        return total_loss_test, outputs_test, target_test.squeeze()

In [ ]:
total_loss_test, outputs_test, target_test = test(model_pred, test_loader)

In [ ]:
print(f"Loss Test: {(sum(total_loss_test) / len(total_loss_test)):.3f}")

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(outputs_test, target_test)